Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-how-to-use-pipeline-drafts.png)

# How to Use Pipeline Drafts
In this notebook, we will show you how you can use Pipeline Drafts. Pipeline Drafts are mutable pipelines which can be used to submit runs and create Published Pipelines.

## Prerequisites and AML Basics
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration Notebook](https://aka.ms/pl-config) first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc.

### Initialization Steps

In [ ]:
import azureml.core
from azureml.core import Workspace
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

### Compute Target
Retrieve an already attached Azure Machine Learning Compute to use in the Pipeline.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
aml_compute_target = "cpu-cluster"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("Found existing compute target: {}".format(aml_compute_target))
except ComputeTargetException:
    print("Creating new compute target: {}".format(aml_compute_target))
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

### Build a Pipeline
Build a simple pipeline to use to create a PipelineDraft.

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

source_directory = "publish_run_train"

train_step = PythonScriptStep(
    name="Training_Step",
    script_name="train.py", 
    compute_target=aml_compute_target, 
    source_directory=source_directory)
print("train step created")

pipeline = Pipeline(workspace=ws, steps=[train_step])
print ("Pipeline is built")

### Create a Pipeline Draft
Create a PipelineDraft by specifying a name, description, experiment_name and Pipeline. You can also specify tags, properties and pipeline_parameter values.

In this example we use the previously created Pipeline object to create the Pipeline Draft. You can also create a Pipeline Draft from an existing Pipeline Run, Published Pipeline, or other Pipeline Draft.

In [ ]:
from azureml.pipeline.core import PipelineDraft

pipeline_draft = PipelineDraft.create(ws, name="TestPipelineDraft",
                                      description="draft description",
                                      experiment_name="pipeline_draft_sample",
                                      pipeline=pipeline,
                                      continue_on_step_failure=True,
                                      tags={'dev': 'true'},
                                      properties={'train': 'value'})

created_pipeline_draft_id = pipeline_draft.id

### List Pipeline Drafts in a Workspace
Use the PipelineDraft.list() function to list all PipelineDrafts in a Workspace. You can use the optional tags parameter to filter on specified tag values.

In [ ]:
pipeline_drafts = PipelineDraft.list(ws, tags={'dev': 'true'})

for pipeline_draft in pipeline_drafts:
    print(pipeline_draft)

### Get a Pipeline Draft by Id

In [ ]:
pipeline_draft = PipelineDraft.get(ws, id=created_pipeline_draft_id)

### Update a Pipeline Draft
The update() function of a pipeline draft can be used to update the name, description, experiment name, pipeline parameter assignments, continue on step failure setting and Pipeline associated with the PipelineDraft. 

In [ ]:
new_train_step = PythonScriptStep(
    name="New_Training_Step",
    script_name="train.py", 
    compute_target=aml_compute_target, 
    source_directory=source_directory)

new_pipeline = Pipeline(workspace=ws, steps=[new_train_step])

pipeline_draft.update(name="UpdatedPipelineDraft", description="has updated train step", pipeline=new_pipeline)

### Submit a Pipeline Run from a Pipeline Draft
Use the pipeline_draft.submit() function to submit a PipelineRun. After the run is submitted, the PipelineDraft can still be edited and used to submit new runs.

In [ ]:
pipeline_run = pipeline_draft.submit_run()
pipeline_run

### Create a Published Pipeline from a Pipeline Draft
Use the pipeline_draft.publish() function to create a Published Pipeline from the Pipeline Draft. After creating a Published Pipeline, the Pipeline Draft can still be edited and used to create other Published Pipelines.

In [ ]:
published_pipeline = pipeline_draft.publish()
published_pipeline